In [32]:
import os
import open3d as o3d
import numpy as np
import laspy as lp
import open3d.visualization as viz
import time
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans

In [13]:
# example code on how to read a LAS file
las = lp.read('./data/Pole1.las')
points = np.vstack((las.x, las.y, las.z)).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

print(f'sample: {points[0:5]}')
print(f'shape: {points.shape}')
print(f'min_x = {las.header.x_min}, max_x = {las.header.x_max}')
print(f'min_y = {las.header.y_min}, max_y = {las.header.y_max}')
print(f'min_z = {las.header.z_min}, max_z = {las.header.z_max}')

sample: [[1.66019183e+05 4.23100000e-01 9.29000000e-02]
 [1.66019165e+05 4.23100000e-01 1.00700000e-01]
 [1.66019175e+05 3.98100000e-01 9.29000000e-02]
 [1.66019190e+05 4.48100000e-01 9.29000000e-02]
 [1.66019190e+05 4.45000000e-01 9.29000000e-02]]
shape: (61584, 3)
min_x = 166018.81453481654, max_x = 166024.2895349596
min_y = -3.501817226409912, max_y = 3.3481829166412354
min_z = -0.035207200795412064, max_z = 5.505418300628662


In [14]:
def visualize(geoms, capture_filename=''):
    '''
        Helper function to run the Open3D visualizer
        Usage: 

        From the pcd variable above, you can call visualize with:
        visualize([pcd])


        Axis:   x = red
                y = green
                z = blue
    '''
    v = viz.Visualizer()
    v.create_window()
    opt = v.get_render_option()
    opt.show_coordinate_frame = True
    
    for g in geoms:
        v.add_geometry(g)
    
    ctr = v.get_view_control()
    # assumes default of 1920x1080 window
    camera_params = o3d.io.read_pinhole_camera_parameters('./camera_trajectory.json')
    ctr.convert_from_pinhole_camera_parameters(camera_params)

    if capture_filename != '':
        time.sleep(1)
        v.capture_screen_image(capture_filename, True)
    else:
        # press "h" for help when in the visualizer for the commands
        v.run()
    v.destroy_window()

In [39]:
def kmean_solution(las):
    
    points = np.vstack((las.x, las.y, las.z)).transpose()
    points_normalized = points.copy()
    range_x = las.header.x_max - las.header.x_min
    range_y = las.header.y_max - las.header.y_min
    range_z = las.header.z_max - las.header.z_min

    mean_z = (points[:,2]).mean()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Perform K-means clustering
    num_clusters = 2
    kmeans = KMeans(n_clusters=num_clusters)
    x_normalized = [(i- las.header.x_min)/range_x for i in points[:,0]]
    y_normalized = [(i- las.header.y_min)/range_y for i in points[:,1]]
    z_normalized = [5*(i- las.header.z_min)/range_z for i in points[:,2]]

    points_normalized[:,0], points_normalized[:,1], points_normalized[:,2] = x_normalized, y_normalized, z_normalized
    kmeans.fit(points_normalized)

    # Get the cluster labels
    labels = kmeans.labels_
    ground_points_idx = np.where(labels == 0)[0]
 
    return ground_points_idx

In [15]:
def slope_based_solution(las_file):
    points = np.vstack((las_file.x, las_file.y, las_file.z)).transpose()
    origin_points = points.copy()

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Define window size for slope calculation
    window_size = 3
    slope_threshold = 1.56

    # Calculate slopes for each point
    slopes = []
    for i in range(window_size, len(points) - window_size):
        # Fit a plane to the local points
        local_points = points[i - window_size:i + window_size + 1, :]
        centroid = np.mean(local_points, axis=0)
        local_points -= centroid
        _, _, V = np.linalg.svd(local_points)
        normal = V[2]

        # Calculate slope from normal vector
        slope = np.arctan(normal[2] / np.sqrt(normal[0] ** 2 + normal[1] ** 2))
        slopes.append(slope)

    # Find ground points based on slope threshold
    slopes = np.array(slopes)
    if np.any(np.isnan(slopes)) or np.any(np.isinf(slopes)):
        # Remove NaN or infinity values from slopes array
        slopes = slopes[np.isfinite(slopes)]
        points = points[:len(slopes), :]
        
    ground_points_idx = np.where(slopes >= abs(slope_threshold))[0]
    ground_points = origin_points[ground_points_idx]
    z_mean_ground = (ground_points[:,2]).mean()
    ground_points_idx = np.where(origin_points[:,2]< z_mean_ground)[0]
    
    return ground_points_idx

In [16]:
def cv2_inrange(HSV, low_range, high_range):
    # Define lower and uppper limits of what we call "ground"
    range_lo = np.array(low_range, np.uint8) 
    range_hi = np.array(high_range, np.uint8) 

    # Mask image to only select ground
    mask = cv2.inRange(HSV, range_lo, range_hi)
    
    return mask

In [17]:
def inRange_solution(las):    
    R = las.points["red"]
    G = las.points["green"]
    B = las.points["blue"]

    # Make a Numpy image by stacking the RGB values and converting to uint8
    BGR = (np.dstack((B, G, R)) >> 8).astype(np.uint8)
    # Convert to HSV
    HSV = cv2.cvtColor(BGR, cv2.COLOR_BGR2HSV)

    lowran = [0, 0, 40]; highran = [180, 18, 230]
    mask = cv2_inrange(HSV, lowran, highran)

    lowran = [36, 25, 25]; highran = [70, 255,255]
    mask2 = cv2_inrange(HSV, lowran, highran)

    lowran = [5,52,39]; highran = [20,255,200]
    mask3 = cv2_inrange(HSV, lowran, highran)


    tt_mask = mask+mask2+mask3

    index = np.where(tt_mask>0)
    tmp = []
    for i in range(len(index[1])):
        tmp.append([index[1][i]])

    return np.array(tmp)

In [46]:
def ground_detection(pcd, points, ground_points_idx):
    pcd.points = o3d.utility.Vector3dVector(points)

    # RGB, colour everything red first
    pcd.paint_uniform_color([1, 0, 0])
    # colour the ground points dark green
    ground_points = pcd.select_by_index(ground_points_idx)
    print("Ground_points: ", ground_points, ground_points_idx)
    ground_points.paint_uniform_color([0.047, 0.117, 0.050])
    
    o3d.visualization.draw_geometries([ground_points])

    return [pcd, ground_points]

In [47]:
'''
####################################
    Write your solution here
####################################
'''
def solution1(filepath: str) -> list[o3d.geometry.PointCloud]:
    las = lp.read(filepath)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    
    #Solution 1: Using image processing method (cv2_inRange) to find color rangs of grounds.
    ground_points_idx_sol1 = inRange_solution(las)

    pcd = o3d.geometry.PointCloud()
    
    list_points_sol1 = ground_detection(pcd, points, ground_points_idx_sol1)

    return list_points_sol1


In [ ]:
num_samples = 8

for n in range(num_samples + 1):
    geoms_sol1  = solution1(f'data/Pole{n}.las')
    
    # Solution 1
    os.makedirs("solution1", exist_ok=True)
    
    # To save the result images.
    visualize(geoms_sol1, f'./solution1/ground{n}.png')
    
    # To visualize the results of solution1.
    visualize(geoms_sol1)

Ground_points:  PointCloud with 42276 points. [[    3]
 [    8]
 [    9]
 ...
 [87784]
 [87785]
 [87786]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 20787 points. [[    0]
 [    2]
 [    3]
 ...
 [61447]
 [61448]
 [61548]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 165732 points. [[   147]
 [   230]
 [   258]
 ...
 [297360]
 [297361]
 [297362]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for

In [40]:
'''
####################################
    Write your solution here
####################################
'''
def solution2(filepath: str) -> list[o3d.geometry.PointCloud]:
    las = lp.read(filepath)
    points = np.vstack((las.x, las.y, las.z)).transpose()
 
    #Solution 2: Using machine learning method (K-means) to find 2 clusters (ground, others).
    ground_points_idx_sol2 = kmean_solution(las)
    
    pcd = o3d.geometry.PointCloud()
    
    list_points_sol2 = ground_detection(pcd, points, ground_points_idx_sol2)

    return list_points_sol2

In [43]:
num_samples = 8

for n in range(num_samples + 1):
    geoms_sol2  = solution2(f'data/Pole{n}.las')
    
    # Solution 2
    os.makedirs("solution2", exist_ok=True)
    
    # To save the result images.
    visualize(geoms_sol2, f'./solution2/ground{n}.png')
    
    # To visualize the results of solution2.
    visualize(geoms_sol2)
    

/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Ground_points:  PointCloud with 23180 points. [    0     1     2 ... 86651 86652 86653]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Ground_points:  PointCloud with 47678 points. [    0     1     2 ... 61581 61582 61583]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Ground_points:  PointCloud with 11210 points. [24505 24506 24519 ... 48285 48286 48287]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Ground_points:  PointCloud with 50502 points. [    0     1     2 ... 61834 61835 61836]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Ground_points:  PointCloud with 60411 points. [    0     1     2 ... 77125 77126 77127]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [28]:
'''
####################################
    Write your solution here
####################################
'''
def solution3(filepath: str) -> list[o3d.geometry.PointCloud]:
    las = lp.read(filepath)
    points = np.vstack((las.x, las.y, las.z)).transpose()

    #Solution 3: Using image processing method (slope based) to find grounds.
    ground_points_idx_sol3 = slope_based_solution(las)
    
    pcd = o3d.geometry.PointCloud()
    
    list_points_sol3 = ground_detection(pcd, points, ground_points_idx_sol3)

    return list_points_sol3

In [44]:
num_samples = 8

for n in range(num_samples + 1):
    geoms_sol3  = solution3(f'data/Pole{n}.las')
    
    # Solution 3
    os.makedirs("solution3", exist_ok=True)
    
    # To save the result images.
    visualize(geoms_sol3, f'./solution3/ground{n}.png')
    
    # To visualize the results of solution3.
    visualize(geoms_sol3)

Ground_points:  PointCloud with 34527 points. [ 1611  1612  1613 ... 87786 87787 87788]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 36642 points. [    0     1     2 ... 61581 61582 61583]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 93918 points. [     1      2      3 ... 298477 298478 298479]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 46837 points. [  821   823   824 ... 72929 72930 72931]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 16190 points. [  481   482   483 ... 58853 58854 58855]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_points:  PointCloud with 46318 points. [    0     1     2 ... 80174 80175 80176]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Ground_poi